In [1]:
%cd ~/work/deeplearning/

/home/asaeed9/work/deeplearning


In [2]:
from theano.sandbox import cuda
#cuda.use('gpu0')
%matplotlib inline
from __future__ import print_function, division
#path="../data/2cat/sample"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink
from keras.preprocessing import image, sequence
import os, sys, cv2
from shutil import copyfile, move

####
from keras.layers.convolutional import *
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device 1 failed:
Bad device number 1. Only 1 devices available.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 5105)
Using Theano backend.


In [3]:
data_dir="/home/asaeed9/work/data/2cat"
path="/home/asaeed9/work/data/2cat/sample/"
results_path = "/home/asaeed9/work/data/2cat/sample/results"
test_path = path + '/test/' #We use all the test data
%cd ../data/2cat/train

/home/asaeed9/work/data/2cat/train


In [4]:
#grey images - minimal image augmentation

#clean previous data
def adjust_prev_data():
    %mv $data_dir/valid/* $data_dir/train/
    
    %mv $path/train/cats/* $data_dir/train/
    %mv $path/train/dogs/* $data_dir/train/
    %mv $path/valid/cats/* $data_dir/train/
    %mv $path/valid/dogs/* $data_dir/train/

#move training images
def move_from_test(new_trainset, path, train, validation):
    print(new_trainset[:10])
    
    
    #clean previous data
#     os.chdir(path + 'test')
    #adjust_prev_data()
    
    #build validation set
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(validation): os.rename(shuf[i], '../valid/' + shuf[i]) 
    
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(train): os.rename(shuf[i], '../sample/train/' + shuf[i]) 
#     %mv ../sample/train/cat*.jpg ../sample/train/cats/
#     %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
#     %cd ../valid

#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(validation): os.rename(shuf[i], '../sample/valid/' + shuf[i]) 
#     %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
#     %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
#     %cd $data_dir/train    
    
#copy training images
def copy_samples(train, validation):
    
    #clean previous data
    adjust_prev_data()
    
    #build validation set
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): os.rename(shuf[i], '../valid/' + shuf[i]) 
    
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(train): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
    %mv ../sample/train/cat*.jpg ../sample/train/cats/
    %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
    %cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(validation): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
    %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
    %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
    %cd $data_dir/train

def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

def pred_batch(imgs, classes):
    preds = model.predict(imgs)
    idxs = np.argmax(preds, axis=1)

    print('Shape: {}'.format(preds.shape))
    print('First 5 classes: {}'.format(classes[:5]))
    print('First 5 probabilities: {}\n'.format(preds[:5]))
    print('Predictions prob/class: ')
    
    for i in range(len(idxs)):
        idx = idxs[i]
        print ('  {:.4f}/{}'.format(preds[i, idx], classes[idx]))


def generate_size_graph(fig_no, training_size, accuracy, loss, start_size, end_size):
    plt.figure(fig_no,figsize=(7,5))
    plt.plot(training_size,accuracy)
    plt.plot(training_size,loss)
    plt.xlabel('Training Size')
    plt.ylabel('Accuracy/Loss')
    plt.title('Training Size vs Accuracy/Loss')
    plt.grid(True)
    plt.legend(['Accuracy','Loss'])
    plt.style.use(['classic'])
    plt.show()
    plt.savefig(path + '/batch_graphs/' +  str(start_size) + '_' + str(end_size) + '.jpg') 
        
def generate_graph(fig_no, epochs, train, val, label, train_title, val_title, train_size):
    plt.figure(fig_no,figsize=(7,5))
    plt.plot(epochs,train)
    plt.plot(epochs,val)
    plt.xlabel('num of Epochs')
    plt.ylabel(label)
    plt.title(train_title + ' vs ' + val_title + '( Samples:' + str(train_size) + ')')
    plt.grid(True)
    plt.legend(['train','val'])
    plt.style.use(['classic'])
    plt.show()
    plt.savefig(results_path + '/batch_graphs/' +  label + '_' + str(train_size) + '.jpg') 
    
def get_train_model(tr_batches, val_batches, epoch):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3, 256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
#             Convolution2D(64,3,3, activation='relu'),
#             BatchNormalization(axis=1),
            #MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Flatten(),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model 

def get_test_model():
    model = Sequential([
                BatchNormalization(axis=1, input_shape=(3,256,256)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
    #             Convolution2D(64,3,3, activation='relu'),
    #             BatchNormalization(axis=1),
                #MaxPooling2D((3,3)),
                Convolution2D(64,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Convolution2D(32,3,3, activation='relu'),
                BatchNormalization(axis=1),
                MaxPooling2D((3,3)),
                Flatten(),
                Dense(1024, activation='relu'),
                BatchNormalization(),
                Dense(2, activation='softmax')
            ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])  
    
    return model

def fit(path, results_path, nepoch, batch_size, train_size, valid_size):
    gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

    #for train in training_range:
    model = None
    copy_samples(train_size, valid_size)

    tr_batches = gen_t.flow_from_directory(path + 'train', batch_size=batch_size)
    val_batches = gen_t.flow_from_directory(path + 'valid', class_mode='categorical', shuffle=True, batch_size=batch_size * 2)

    model = get_train_model(tr_batches, val_batches, nepoch)
    model.save_weights(results_path+ '/' + 'ft_' + str(train_size) + '.e' + str(nepoch))

def predict(path, model):
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+'test', class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    probs = model.predict(test_data)
    
    #loss_score.append(score[0])
    #accuracy_score.append(score[1])
    
    print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))
    print("\nProbabilities:{}".format(probs))
    return probs, test_batches

In [5]:
%cd $data_dir/train

nepoch = 5
batch_size = 16
train_size = 100
valid_size = int(math.floor(.2 * train_size))
print('sample size: {}'.format(train_size))


#for i in range(10):
    
#model = fit(path, results_path, nepoch, batch_size, train_size, valid_size)

model = None
model = get_test_model() 

model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(nepoch))
probs, test_batches = predict(path, model)

retrain_idx = np.unique(np.where(np.logical_and(probs >=0.4, probs<=0.6))[0])
retrain_set = [test_batches.filenames[i] for i in retrain_idx]


# #copy all test files that meet some criteria to train/valid directory

# os.chdir(path + 'test')
# ndog = sum('dog' in name for name in retrain_set)
# ncat =  sum('cat' in name for name in retrain_set)
# limit = min(ncat, ndog)

# #remove all train/validation images

# print("Images in retrain set: {}".format(len(retrain_set)))
# cats_copied=5
# dogs_copied=5
# for fil in retrain_set:
#     #print("Retrain File: {}".format(fil))
#     fil_cpy = fil[fil.find('/')+1:]
#     if 'cat' in fil and cats_copied <= to_be_copied:
#         #print(os.path.join(path + "test/cats/"+ fil_cpy))
#         #print(os.path.join(path + "test_copy/"+ fil_cpy))
#         copyfile(os.path.join(path + "test/cats/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#         cats_copied+=1
#     elif 'dog' in fil and dogs_copied <= to_be_copied:
#         #print(os.path.join(path + "test/dogs/"+ fil_cpy))
#         #print(os.path.join(path + "test_copy/"+ fil_cpy))              
#         copyfile(os.path.join(path + "test/dogs/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#         dogs_copied+=1
    
#     if cats_copied >= to_be_copied and dogs_copied >= to_be_copied:
#         #print(len(os.listdir(path + "test_copy/")))
#         print(cats_copied, dogs_copied, to_be_copied)
#         break

# print("copy done.")

#move to validation directory
#replenish validation set

#move_to_test(cats_copied+dogs_copied)

def move_samples(retrain_set,dest_path, n):
    retrain_list = list(retrain_set)
    for fil in range(n):
        fil = retrain_list.pop()
        fil_cpy = fil[fil.find('/')+1:]

        if 'cat' in fil_cpy and cats_copied <= limit:
            copyfile(os.path.join(path + "test/cats/"+ fil_cpy), os.path.join(path + dest_path + "/cats/"+ fil_cpy))
            cats_copied+=1
        elif 'dog' in fil_cpy and dogs_copied <= limit:
            copyfile(os.path.join(path + "test/dogs/"+ fil_cpy), os.path.join(path + dest_path + "/dogs/" + fil_cpy))
            dogs_copied+=1

#         if cats_copied >= limit and dogs_copied >= limit:
#             #print(len(os.listdir(path + "test_copy/")))
#             print(cats_copied, dogs_copied, n)
#             break

        
        
        
        
        
        
        
        
    
def move_to_train(retrain_set, limit):
    cats_copied = 0
    dogs_copied = 0
    valid_limit = int(math.floor(.2*(limit*2)))    

    move_samples(retrain_set, "test_copy", valid_limit)
    
#     for fil in retrain_set:
#         fil_cpy = fil[fil.find('/')+1:]
#         if 'cat' in fil and cats_copied <= limit:
#             #print(os.path.join(path + "test/cats/"+ fil_cpy))
#             #print(os.path.join(path + "test_copy/"+ fil_cpy))
#             copyfile(os.path.join(path + "test/cats/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#             cats_copied+=1
#         elif 'dog' in fil and dogs_copied <= limit:
#             #print(os.path.join(path + "test/dogs/"+ fil_cpy))
#             #print(os.path.join(path + "test_copy/"+ fil_cpy))              
#             copyfile(os.path.join(path + "test/dogs/"+ fil_cpy), os.path.join(path + "test_copy/"+ fil_cpy))
#             dogs_copied+=1

#         if cats_copied >= limit and dogs_copied >= limit:
#             #print(len(os.listdir(path + "test_copy/")))
#             print(cats_copied, dogs_copied, limit)
#             break

    
    
    

# def restore_backup(source, destination):
#     %mv $source/cat*.jpg $destination/cats/
#     %mv $source/dog*.jpg $destination/dogs/
    
# def remove_to_backup(source, destination):
#     %mv $source/* $destination/

# remove_to_backup(path + 'train/cats/', path + 'train_bkup/')
# remove_to_backup(path + 'train/dogs/', path + 'train_bkup/')

#restore_backup(path + 'train_bkup', path + 'train')

# nimages = cats_copied + dogs_copied
# def move_to_test(nimages):
#     os.chdir("/home/asaeed9/work/data/2cat/train")
#     g = glob('*.jpg')
#     shuf = np.random.permutation(g)
#     for i in range(nimages): os.rename(shuf[i], '../sample/test/' + shuf[i])
#     os.chdir("../sample/test/")
#     move("cat*.jpg")
#     %mv ../sample/test/cat*.jpg ../sample/cats/
#     %mv ../sample/test/dog*.jpg ../sample/dogs/    




#move_from_test(retrain_set, path, train_size, validation_size)


/home/asaeed9/work/data/2cat/train
sample size: 100
Found 3000 images belonging to 2 classes.


MemoryError: Error allocating 264257536 bytes of device memory (out of memory).
Apply node that caused the error: GpuAllocEmpty(Assert{msg='The convolution would produce an invalid shape (dim[0] < 0).'}.0, Assert{msg='The convolution would produce an invalid shape (dim[1] < 0).'}.0, Assert{msg='The convolution would produce an invalid shape (dim[2] <= 0).'}.0, Assert{msg='The convolution would produce an invalid shape (dim[3] <= 0).'}.0)
Toposort index: 109
Inputs types: [TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(), (), (), ()]
Inputs strides: [(), (), (), ()]
Inputs values: [array(32), array(32), array(254), array(254)]
Outputs clients: [[GpuDnnConv{algo='small', inplace=True}(GpuContiguous.0, GpuContiguous.0, GpuAllocEmpty.0, GpuDnnConvDesc{border_mode='valid', subsample=(1, 1), conv_mode='conv', precision='float32'}.0, Constant{1.0}, Constant{0.0})]]

HINT: Re-running with most Theano optimization dis

In [ ]:
model = None
model = get_test_model()
loss_score = []
accuracy_score = []
training_size = [100]

os.chdir(results_path)
files = filter(os.path.isfile, os.listdir(results_path))
files.sort(key=lambda x: os.path.getmtime(x))

for f in files:
    
    print("\nFile being processed: {}".format(f))
    
    train_size = int(f[f.find('_')+1 : f.find('.')])
    epoch = int(f[f.find('e')+1 : ])

    model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(epoch))
    
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+'test', class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    loss_score.append(score[0])
    accuracy_score.append(score[1])
    
    print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))


In [ ]:
model = None
model = get_test_model()
loss_score = []
accuracy_score = []
training_size = [100]

os.chdir(results_path)
files = filter(os.path.isfile, os.listdir(results_path))
files.sort(key=lambda x: os.path.getmtime(x))

for f in files:
    
    print("\nFile being processed: {}".format(f))
    
    train_size = int(f[f.find('_')+1 : f.find('.')])
    epoch = int(f[f.find('e')+1 : ])

    model.load_weights(results_path+'/ft_' + str(train_size) + '.e' + str(epoch))
    
    gen_test = image.ImageDataGenerator()
    test_batches = gen_test.flow_from_directory(path+'test', class_mode=None, target_size=(256,256), shuffle=False, batch_size=1)
    test_data = np.concatenate([test_batches.next() for i in range(test_batches.nb_sample)])
    test_labels = onehot(test_batches.classes)
    score = model.evaluate(test_data, test_labels)
    
    probs = model.predict(test_data)
    
    loss_score.append(score[0])
    accuracy_score.append(score[1])
    
    print("\nLoss:{}, Accuracy:{}".format(score[0], score[1]))
    print("\nProbabilities:{}".format(probs))


In [ ]:
test_path

In [ ]:
from PIL import Image
Image.open(test_path +  'cats/cat.6013.jpg')


In [ ]:
#probs[:10]
retrain_idx = np.where(np.logical_and(probs >=0.4, probs<=0.6))[0]
retrain_set = [test_batches.filenames[i] for i in retrain_idx]



retrain_set[:10]

In [ ]:
def get_train_model(tr_batches, epoch):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3, 256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
#             Convolution2D(64,3,3, activation='relu'),
#             BatchNormalization(axis=1),
            #MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Flatten(),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    #model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=2)
    #model.optimizer.lr = 0.001
    #model.fit_generator(tr_batches, tr_batches.nb_sample, nb_epoch=epoch - 2)
    return model 

In [ ]:
#model = get_train_model(new_data, nepoch)
#model.load_weights(results_path + "/ft_100.e50")
#model.fit_generator(new_data, len(new_data), nb_epoch = 2)